In [2]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("2c.csv", delimiter=";")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%d/%m/%Y %H:%M")

In [3]:
for i in ["TRIP", "TEMPERATURE", "FUEL", "PERSONS"]:
    df[i].fillna(method="ffill", inplace=True)

C:\Users\canut\AppData\Local\Temp\ipykernel_1096\1881467328.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[i].fillna(method='ffill', inplace=True)


In [6]:
df["ALTITUDE"].fillna(
    df["ALTITUDE"].rolling(window=15, min_periods=1).mean(), inplace=True
)



df["ALTITUDE"] = df["ALTITUDE"].round(0)

In [7]:
dn = list(df.select_dtypes(include=np.number).columns)

for i in dn:
    df[i] = df[i].astype(int)

In [8]:
df[df["ALTITUDE"].isna()].index

Index([], dtype='int64')

In [31]:
for i in [366, 367]:
    df.at[i, "ALTITUDE"] = 106

In [33]:
df.at[387, "ALTITUDE"] = 98

In [9]:
df.to_csv("test_clean_2.csv")

In [46]:
list_t = list(df[df["TIMESTAMP"].notnull()].index)

In [55]:
df[df["TIMESTAMP"].notnull()]

,Unnamed: 0,TIMESTAMP,SPEED,ALTITUDE,TRIP,TEMPERATURE,FUEL,PERSONS
0,0,2024-04-05 18:01:00,5,30.0,GO,21.0,BP,3.0
42,42,2024-04-05 18:18:00,0,46.0,GO,21.0,BP,3.0
43,43,2024-04-05 18:47:00,0,46.0,RETURN,21.0,BP,3.0
101,101,2024-04-05 19:00:00,0,30.0,RETURN,21.0,BP,3.0
102,102,2024-04-06 09:18:00,0,33.0,GO,22.0,BP,3.0
147,147,2024-04-06 10:02:00,0,105.0,GO,22.0,BP,3.0
148,148,2024-04-06 10:15:00,0,105.0,RETURN,26.0,BP,1.0
202,202,2024-04-06 11:27:00,0,33.0,RETURN,26.0,BP,1.0
203,203,2024-04-13 10:31:00,0,33.0,GO,19.0,BP,2.0
286,286,2024-05-14 13:50:00,0,105.0,GO,22.0,REPSOL,4.0


In [150]:
df

,Unnamed: 0,TIMESTAMP,SPEED,ALTITUDE,TRIP,TEMPERATURE,FUEL,PERSONS
0,0,2024-04-14 16:06:00,0,105,RETURN,23,REPSOL,2
1,1,NaT,4,105,RETURN,23,REPSOL,2
2,2,NaT,8,105,RETURN,23,REPSOL,2
3,3,NaT,3,106,RETURN,23,REPSOL,2
4,4,NaT,3,106,RETURN,23,REPSOL,2
...,...,...,...,...,...,...,...,...
218,218,NaT,72,29,RETURN,23,REPSOL,1
219,219,NaT,12,29,RETURN,23,REPSOL,1
220,220,NaT,3,29,RETURN,23,REPSOL,1
221,221,NaT,2,29,RETURN,23,REPSOL,1


In [149]:
df = pd.read_csv("test_clean_2.csv", delimiter=",")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%Y-%m-%d %H:%M:%S")

df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="%d/%m/%Y %H:%M:%S")

In [151]:
lr = df[df["TIMESTAMP"].notnull()].index
lr

Index([0, 95, 96, 165, 166, 222], dtype='int64')

In [135]:
def get_even_lists(lst):
    even_pos = []
    odd_pos = []
    lim = len(lst)

    for i in range(len(lst)):
        if i % 2 == 1:
            odd_pos.append(lst[i])
        else:
            even_pos.append(lst[i])
    return odd_pos, even_pos

In [152]:
odd_l, even_l = get_even_lists(lr)
odd_l, even_l

([95, 165, 222], [0, 96, 166])

In [153]:
lg = []
for i, j in zip(even_l, odd_l):
    l_i = pd.date_range(
        start=pd.to_datetime(str(df["TIMESTAMP"][i])),
        end=pd.to_datetime(str(df["TIMESTAMP"][j])),
        periods=(j - i + 1),
    ).round(freq="min")
    print(l_i[-1])
    lg.append(l_i)

ld = [x for xs in lg for x in xs]
df["TIMESTAMP"] = ld

2024-04-14 16:59:00
2024-04-19 16:18:00
2024-04-19 16:57:00


In [154]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [155]:
df.head()

,TIMESTAMP,SPEED,ALTITUDE,TRIP,TEMPERATURE,FUEL,PERSONS
0,2024-04-14 16:06:00,0,105,RETURN,23,REPSOL,2
1,2024-04-14 16:07:00,4,105,RETURN,23,REPSOL,2
2,2024-04-14 16:07:00,8,105,RETURN,23,REPSOL,2
3,2024-04-14 16:08:00,3,106,RETURN,23,REPSOL,2
4,2024-04-14 16:08:00,3,106,RETURN,23,REPSOL,2


In [156]:
df_2 = df.copy()

In [160]:
df_2.shape

(223, 7)

In [162]:
df_glo.shape

(629, 7)

In [161]:
df_glo = pd.concat([df_1, df_2], axis=0)

In [163]:
df_glo.to_csv("final_1.csv")